In [1]:
from pathlib import Path

import numpy as np

from vgn.io import *
from vgn.perception import *
from vgn.assign_grasp_affordance import affrdnce_label_dict
from vgn.utils.implicit import semantic_label_dict

In [2]:
root = Path('/work/scratch/sj93qicy/potato-net/data/3DGraspAff/data_affnet_train_random_raw_GPG_60')
df = read_df_with_surface_clouds(root)

In [3]:
# Grasp label balance
positives = df[df["label"] == 1]
negatives = df[df["label"] == 0]
print("Positives: {}, Negatives: {}".format(len(positives), len(negatives)))

Positives: 967777, Negatives: 967650


In [4]:
# Affordance label balance
# only the positives have an affordance label
for aff_key in affrdnce_label_dict.keys():
    affrdnce_positives = positives[positives[aff_key] == 1]
    affrdnce_negatives = positives[positives[aff_key] == 0]
    print("Affordance: {}, Positives: {}, Negatives: {}, Ratio: {}".format(
        aff_key, len(affrdnce_positives), len(affrdnce_negatives), len(affrdnce_positives) / len(affrdnce_negatives)))


Affordance: handover, Positives: 525106, Negatives: 442671, Ratio: 1.1862218216237341
Affordance: cut, Positives: 157387, Negatives: 810390, Ratio: 0.19421142906501807
Affordance: stab, Positives: 157387, Negatives: 810390, Ratio: 0.19421142906501807
Affordance: lift, Positives: 50334, Negatives: 917443, Ratio: 0.0548633539086352
Affordance: wrap, Positives: 103662, Negatives: 864115, Ratio: 0.11996319934268009
Affordance: pour, Positives: 261693, Negatives: 706084, Ratio: 0.3706258745418392
Affordance: wear, Positives: 70935, Negatives: 896842, Ratio: 0.07909419942420182
